In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as Function
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image

# Load data

In [2]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size)

# Create Softmax Layer



In [100]:

input_data=[]
target_data=[]

for _, (data,target) in enumerate(train_loader):
        data =data.view(-1,784)
        data=data.numpy()
        input_data.append(data)
        
        target=target.view(-1)
        target=target.numpy()
        target_data.append(target)

n=np.array(target_data[0])
x=torch.from_numpy(n)

for i in range(1,938):
    y=np.array(target_data[i])
    y=torch.from_numpy(y)
    x=torch.cat((x, y),0)
    

full_input=np.array(input_data[0])
full_input=torch.Tensor(full_input)

for i in range(1,938):
    intermed=np.array(input_data[i])
    intermed=torch.Tensor(intermed)
    
    full_input=torch.cat((full_input, intermed),0)

x=x.numpy()

target_array=np.zeros((60000,10))

count=0
for n in x:
    target_array[count][n]=1
    count+=1

target_array=torch.Tensor(target_array)

final_training_data=torch.cat((full_input, target_array), 1)

# Train

In [101]:
class RBM(nn.Module):
    def __init__(self,
                 n_vis=794,
                 n_hin=625,
                 k=1):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k
    
    def sample_p(self,p):
        return Function.relu(torch.sign(p - Variable(torch.rand(p.size()))))
    
    def v_to_h(self,v):
        p_h = Function.sigmoid(Function.linear(v,self.W,self.h_bias))
        sample_h = self.sample_p(p_h)
        return p_h,sample_h
    
    def h_to_v(self,h):
        p_v = Function.sigmoid(Function.linear(h,self.W.t(),self.v_bias))
        sample_v = self.sample_p(p_v)
        return p_v,sample_v
        
    def forward(self,v):
        pre_h1,h1 = self.v_to_h(v)
        
        h_ = h1
        
        for _ in range(self.k):
            pre_v_,v_ = self.h_to_v(h_)
            pre_h_,h_ = self.v_to_h(v_)
        
        return v,v_, h_
    
    def free_energy(self,v):
        vbias_term = v.mv(self.v_bias)
        wx_b = Function.linear(v,self.W,self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (-hidden_term - vbias_term).mean()

In [102]:
rbm = RBM(k=10)
train_op = optim.SGD(rbm.parameters(),0.1)

In [103]:
batch_size_=64

for epoch in range(10):
    loss_ = []
    reconstruction_error=0
    s=0
    for n in range(0, len(full_input)- batch_size_, batch_size_):
        sample_data=final_training_data[n:n+batch_size_]
        sample_data=Variable(sample_data)
        sample_data = sample_data.bernoulli()
        
        
        v,v1,h1 = rbm(sample_data)
        
        loss = rbm.free_energy(v) - rbm.free_energy(v1)
        loss_.append(loss.data[0])
        
        train_op.zero_grad()
        loss.backward()
        train_op.step()
        
        reconstruction_error+=torch.mean(torch.abs(v-v1))
        s+=1
        
    print ('Epoch: ' + str(epoch+1) +  
           ' - reconstructions error:  ' + str(reconstruction_error.data.numpy()[0]/s) + 
           ' - free energy loss: ' + str(np.mean(loss_)))  

Epoch: 1 - reconstructions error:  0.1368513891 - free energy loss: 2.49504118668
Epoch: 2 - reconstructions error:  0.100497438152 - free energy loss: -3.63298811449
Epoch: 3 - reconstructions error:  0.0922493379933 - free energy loss: -2.39830875295
Epoch: 4 - reconstructions error:  0.0880711981937 - free energy loss: -1.4871570176
Epoch: 5 - reconstructions error:  0.0852940822996 - free energy loss: -0.783779115789
Epoch: 6 - reconstructions error:  0.0832473445473 - free energy loss: -0.315411025203
Epoch: 7 - reconstructions error:  0.0817952517384 - free energy loss: 0.101648906761
Epoch: 8 - reconstructions error:  0.0805686616847 - free energy loss: 0.312100047108
Epoch: 9 - reconstructions error:  0.0794975663453 - free energy loss: 0.540013886312
Epoch: 10 - reconstructions error:  0.0787476058195 - free energy loss: 0.73954634529


# Testing

In [104]:
input_data=[]

for _, (data,target) in enumerate(test_loader):
        data =data.view(-1,784)
        data=data.numpy()
        input_data.append(data)
        



In [105]:
full_input=np.array(input_data[0])
full_input=torch.Tensor(full_input)

for i in range(1,157):
    intermed=np.array(input_data[i])
    intermed=torch.Tensor(intermed)
    
    full_input=torch.cat((full_input, intermed),0)      

In [106]:
test_dummy=np.zeros((len(full_input), 10))
test_dummy=torch.Tensor(test_dummy)

In [107]:
test_set=torch.cat((full_input, test_dummy), 1)

In [108]:
import time
start_time = time.time()

output=[]
target_output=[]

test_loss = 0
s=0
for n in range(0,len(test_set)):
    sample_data=Variable(test_set)
    sample_data = sample_data[n:n+1]

    v,v1,h1 = rbm(sample_data)
    test_loss+=torch.mean(torch.abs(v-v1))
    s+=1
    
    loss = rbm.free_energy(v) - rbm.free_energy(v1)
    loss_.append(loss.data[0])
    
    output.append(v1)
    target_output.append(v)
    
    
print ('Reconstructions error:  ' + str(test_loss.data.numpy()[0]/s)+ 
          ' - free energy loss: ' + str(np.mean(loss_)))  
print("--- training time is %s seconds ---" % (time.time() - start_time))

Reconstructions error:  0.0837298339844 - free energy loss: 16.8534106355
--- training time is 68.39816904067993 seconds ---


# Arrange output array

In [126]:
target_data=[]

for _, (data,target) in enumerate(test_loader):
    target=target.view(-1)
    target=target.numpy()
    target_data.append(target)

n=np.array(target_data[0])
target_1=torch.from_numpy(n)

for i in range(1,157):
    y=np.array(target_data[i])
    y=torch.from_numpy(y)
    target_1=torch.cat((target_1, y),0)

In [127]:
target_1=target_1.numpy()

In [128]:
output_array=[]

for n in range(0, 10000):
    output_n=output[n][0].data.numpy()
    output_array.append(output_n)
    
output_array=np.array(output_array)

In [129]:
digits=[]

for n in range(0, 10000):
    digit=output_array[n][784:]
    digits.append(digit)

# Classification

In [130]:
import pandas as pd

In [131]:
def getdigit(outputarray):
    results=[]
    index=[]
    for i in range(0,len(outputarray)):
        for n in range(0, 10):
            if outputarray[i][n]==1:
                results.append(int(n))
                index.append(int(i))
        if all(outputarray[i][0:10]==0):
            results.append(int(0))
            index.append(int(i))
   # results=np.array(results)
    #index=np.array(index)
    #total=np.stack((index, results), axis=0)
    total=list(zip(index,results))
    return total

In [132]:
output_digits=getdigit(digits)
#output_digits=results

In [133]:
classification=[]
for i, x in enumerate(output_digits):
    classification.append(x)

In [134]:
classification=np.array(classification)

In [135]:
df_class=pd.DataFrame(classification)

final_class=df_class.drop_duplicates(subset=0, keep='first')

final_class=final_class.reset_index()

correct=0

for i in range(0, 10000):
    if target_1[i]==final_class[1][i]:
        correct+=1
        
accuracy=correct/len(target_1)

print ( 'classification accuracy is %s ' %accuracy)

classification accuracy is 0.8596 


# Compress

In [136]:
weights=rbm.W
weights=weights.data.numpy()


x=weights.reshape(496250)

In [137]:
savedweights=weights
x=savedweights.reshape(496250)

In [122]:
# Mean

positive=[]
negative=[]
for n in x:
    if n>0:
        positive.append(n)
    elif n<0:
        negative.append(n)

m=[]
for n in negative:
    if n>-0.5:
        m.append(n)

m=np.array(m)
print (np.median(m))

-0.064853


In [124]:
# Prune


i = []

for w in x:
    if w < 0.03 and w > -0.03:
        w=0
        
    else:
        w=w
    i.append(w)

i=np.array(i).reshape(625, 794)

w=torch.Tensor(i)


o=torch.nn.Parameter(w)
rbm.W=o

In [ ]:
# quantise

q=[]

for w in x:
    if w > 0.15:
        w=float(1)
    elif w<-0.15:
        w=float(-1)
    else:
        w=float(0)
    q.append(w)
    
q=np.array(q).reshape(625, 784)
w=torch.Tensor(q)
w=torch.nn.Parameter(w)
rbm.W=w

In [ ]:
# mean quantise

q=[]

for w in x:
    if w >= 0.3:
        w=float(1)
    elif w > 0.02 and w<0.3:
        w=float(0.07)
    elif w<=-0.3:
        w=float(-1)
    elif w<-0.02 and w>-0.3:
        w=float(-0.07)
    else:
        w=float(0)
    q.append(w)
    
q=np.array(q).reshape(1000, 784)
w=torch.Tensor(q)
w=torch.nn.Parameter(w)
rbm.W=w

In [ ]:
import time
start_time = time.time()

output=[]
target_output=[]

test_loss = 0
s=0
for n in range(0,len(test_set)):
    sample_data=Variable(test_set)
    sample_data = sample_data[n:n+1]

    v,v1,h1 = rbm(sample_data)
    test_loss+=torch.mean(torch.abs(v-v1))
    s+=1
    
    loss = rbm.free_energy(v) - rbm.free_energy(v1)
    loss_.append(loss.data[0])
    
    output.append(v1)
    target_output.append(v)
    
    
print ('Reconstructions error:  ' + str(test_loss.data.numpy()[0]/s)+ 
          ' - free energy loss: ' + str(np.mean(loss_)))  
print("--- training time is %s seconds ---" % (time.time() - start_time))

In [125]:
import time
start_time = time.time()

output=[]
target_output=[]

test_loss = 0
s=0
for n in range(0,len(test_set)):
    sample_data=Variable(test_set)
    sample_data = sample_data[n:n+1]

    v,v1,h1 = rbm(sample_data)
    test_loss+=torch.mean(torch.abs(v-v1))
    s+=1
    
    loss = rbm.free_energy(v) - rbm.free_energy(v1)
    loss_.append(loss.data[0])
    
    output.append(v1)
    target_output.append(v)
    
    
print ('Reconstructions error:  ' + str(test_loss.data.numpy()[0]/s)+ 
          ' - free energy loss: ' + str(np.mean(loss_)))  
print("--- training time is %s seconds ---" % (time.time() - start_time))

Reconstructions error:  0.0838052612305 - free energy loss: 17.6841711024
--- training time is 75.55067801475525 seconds ---
